In [1]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
#from .utils import PARAMETER_NAMES_ALL_PRECESSINGBNS_BILBY
from river.models.utils import *
from river.data.utils import *

import river.data
from river.data.datagenerator import DataGeneratorBilbyFD
from river.data.dataset import DatasetSVDStrainFDFromSVDWFonGPU #DatasetStrainFD, #DatasetStrainFDFromPreCalSVDWF
#import river.data.utils as datautils
from river.data.utils import *

from river.models import embedding
from river.models.utils import *
#from river.models.embedding.pca import project_strain_data_FDAPhi
from river.models.embedding.conv import EmbeddingConv1D, EmbeddingConv2D
from river.models.embedding.mlp import EmbeddingMLP1D

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
import numpy as np
import bilby 
import pycbc 
import sys
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

import pickle
#from sklearn.utils.extmath import randomized_svd
import sklearn
from sklearn.decomposition import IncrementalPCA, randomized_svd, KernelPCA
import sklearn.decomposition 

import glob
import time

In [3]:
validfolder = '/home/qian.hu/mlpe/training_data/bns_50Hz1024Hz32s_lowspin_lb/valid'
trainfolder = '/home/qian.hu/mlpe/training_data/bns_50Hz1024Hz32s_lowspin_lb/train'

valid_filenames = glob.glob(f"{validfolder}/batch*/*.h5")
train_filenames = glob.glob(f"{trainfolder}/batch*/*.h5")



In [4]:
source_type = 'BNS'
detector_names = ['H1', 'L1', 'V1'] 
duration = 32
f_low = 50
f_high = 1024

f_ref = 20
sampling_frequency = 2048
waveform_approximant = 'IMRPhenomPv2_NRTidal'
parameter_names = PARAMETER_NAMES_ALL_PRECESSINGBNS_BILBY
PSD_type = 'bilby_default' #'zero_noise' bilby_default
use_sealgw_detector = True

Vhfile="/home/qian.hu/mlpe/river/test/outputs/Vh_50Hz1024Hz32s.pickle"

data_generator = DataGeneratorBilbyFD(source_type,
            detector_names, 
            duration, 
            f_low, 
            f_ref, 
            sampling_frequency, 
            waveform_approximant, 
            parameter_names,
            PSD_type=PSD_type,
            use_sealgw_detector=use_sealgw_detector,
            f_high=f_high)


18:14 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_neutron_star
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


Using bilby_default PSDs to generate data.


In [5]:
device='cuda'

dataset = DatasetSVDStrainFDFromSVDWFonGPU(valid_filenames, PARAMETER_NAMES_CONTEXT_PRECESSINGBNS_BILBY, data_generator,
                                     Nbasis=512, Vhfile=Vhfile, device=device)

batch_size = 4
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [7]:
theta, x = dataset[1]

for theta_batch, x_batch in dataloader:
    pass

In [8]:
x_batch.shape

torch.Size([4, 6, 512])

In [32]:
#embd = EmbeddingConv1D(ndet=3, nout=256, num_blocks=2, use_psd = False).to(device)


In [33]:
x.shape

torch.Size([6, 512])

In [34]:
embd.nchannel

512

In [39]:
condition = embd(x.unsqueeze(0))

In [225]:
(1,2) == (2,1)

False

In [243]:
from torch import nn
import torch.nn.functional as F

class ResidualConvBlock2D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1):
        super().__init__()
        #self.padding=padding
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=1, padding=0, dilation=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        
        if in_channels != out_channels or kernel_size!=1 or stride!=1 or padding!=0 or dilation!=1:
            self.downsample = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, dilation=dilation) 
        else:
            self.downsample = nn.Identity()

        '''
        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1),
                nn.BatchNorm2d(out_channels)
            )
        '''

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.downsample(residual)
        out = F.relu(out)
        return out

class MyEmbeddingConv2D(nn.Module):
    def __init__(self, nout, **kwargs):
        super().__init__()

        self.nout = nout
        self.layers = nn.ModuleList([self.make_layer()])
        self.linear = nn.Linear(3, self.nout)

    def make_layer(self):
        layers = []
        layers.append(ResidualConvBlock2D(in_channels=1, out_channels=8,
                                          kernel_size=(3,2), stride=(1,1), dilation=(1,1), padding = (2,0)))
        layers.append(ResidualConvBlock2D(in_channels=8, out_channels=16,
                                          kernel_size=(3,4), stride=(1,1), dilation=(2,1), padding = (2,0)))
        layers.append(ResidualConvBlock2D(in_channels=16, out_channels=16,
                                          kernel_size=(3,2), stride=(1,1), dilation=(2,1), padding = (2,0)))
        layers.append(ResidualConvBlock2D(in_channels=16, out_channels=3,
                                          kernel_size=(3,2), stride=(1,1), dilation=(1,1), padding = (2,0)))
        return nn.Sequential(*layers)

    def forward(self, x):
        # x : [batch_size, channel (det_123, amp/phase) = 2*ndet, length (number of samples)]
        # bs,_,_,_  = x.shape
        x = x.unsqueeze(1)
        for layer in self.layers:
            x = layer(x)
        x = F.avg_pool2d(x, x.size()[2:])
        x = x.view(x.size(0), -1)
        output = self.linear(x)
        return output

In [147]:
x_test = x_batch.unsqueeze(1)

In [148]:
x_test.shape

torch.Size([4, 1, 6, 512])

In [244]:
embd = MyEmbeddingConv2D(nout=256).to(device)


In [245]:
count_parameters(embd)

8726

In [227]:
torch.nn.Conv2d?

In [165]:
layer1 = torch.nn.Conv2d(1, 8, kernel_size=(3,2), stride=(1,1), dilation=(1,1), padding = (2,0)).to(device)

In [220]:
layer1 = ResidualConvBlock2D(1, 8, kernel_size=(3,2), stride=(4,2), dilation=(3,4), padding = (3,2)).to(device)

In [246]:
a = embd(x_batch)

In [247]:
a.shape

torch.Size([4, 256])

In [188]:
import glasflow

In [190]:
glasflow.nflows.nn.nets.ConvResidualNet??

In [146]:
nn.Conv2d?

In [138]:
condition = embd(x_batch.unsqueeze(1))

RuntimeError: The size of tensor a (255) must match the size of tensor b (512) at non-singleton dimension 3

In [110]:
condition.shape

torch.Size([4, 256])

In [133]:
F.avg_pool2d?

In [144]:
aa = torch.tensor([[[1+1j, 2, 3, 4, 5, 6, 7]]], dtype=torch.complex64)
F.avg_pool1d(aa, kernel_size=3, stride=2)

RuntimeError: "avg_pool2d" not implemented for 'ComplexFloat'